In [4]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

from KETIPrePartialDataPreprocessing import main

inputType ='file' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}

imputation_param ={ "imputation_method":
    [{"min":0,"max":1,"method":"linear"}, {"min":2,"max":3,"method":"spline"}],"totalNanLimit":70
}
    
# ffill~polynomial_interpolate=> 각 구간마다 딱 고만큼만 Fill

input_data = main.inputControl(inputType)
from KETIPrePartialDataPreprocessing import data_preprocessing
import numpy as np
input_test_data = input_data.replace(4.0, np.nan)
output = data_preprocessing.ByAllMethod(input_test_data[:25], refine_param, outlier_param, imputation_param)

data Length after removeDuplication: 25
data Length after make_static_frequency: 25
NaN Ratio: temp 52.0 %
linear
[(Timestamp('2021-01-28 12:53:00'), 3), (Timestamp('2021-01-28 12:57:00'), 8), (Timestamp('2021-01-28 13:14:00'), 2)]
before masking: temp    8
dtype: int64 temp
after masking: temp    10
dtype: int64
                     temp
timedate                 
2021-01-28 12:51:00   3.0
2021-01-28 12:52:00   3.0
2021-01-28 12:53:00   3.0
2021-01-28 12:54:00   NaN
2021-01-28 12:55:00   NaN
2021-01-28 12:56:00   3.0
2021-01-28 12:57:00   3.0
2021-01-28 12:58:00   NaN
2021-01-28 12:59:00   NaN
2021-01-28 13:00:00   NaN
2021-01-28 13:01:00   NaN
2021-01-28 13:02:00   NaN
2021-01-28 13:03:00   NaN
2021-01-28 13:04:00   NaN
2021-01-28 13:05:00   3.0
2021-01-28 13:06:00   3.0
2021-01-28 13:07:00   3.0
2021-01-28 13:08:00   3.0
2021-01-28 13:09:00   3.0
2021-01-28 13:10:00   3.0
2021-01-28 13:11:00   3.0
2021-01-28 13:12:00   3.0
2021-01-28 13:13:00   3.0
2021-01-28 13:14:00   3.0
2021-01-2

In [5]:
output

{'original':                      temp  co2    pm10
 timedate                              
 2021-01-28 12:51:00   3.0  5.0 -9999.0
 2021-01-28 12:52:00   3.0  5.0    64.0
 2021-01-28 12:53:00   NaN  6.0    56.0
 2021-01-28 12:54:00   NaN  7.0    59.0
 2021-01-28 12:55:00   NaN  7.0    59.0
 2021-01-28 12:56:00   3.0  6.0    58.0
 2021-01-28 12:57:00   NaN  7.0    57.0
 2021-01-28 12:58:00   NaN  7.0    58.0
 2021-01-28 12:59:00   NaN  6.0    57.0
 2021-01-28 13:00:00   NaN  7.0    56.0
 2021-01-28 13:01:00   NaN  6.0    58.0
 2021-01-28 13:02:00   NaN  7.0    57.0
 2021-01-28 13:03:00   NaN  6.0    56.0
 2021-01-28 13:04:00   NaN  6.0    57.0
 2021-01-28 13:05:00   3.0  6.0    59.0
 2021-01-28 13:06:00   3.0  6.0    61.0
 2021-01-28 13:07:00   3.0  6.0    59.0
 2021-01-28 13:08:00   3.0  6.0    61.0
 2021-01-28 13:09:00   3.0  5.0    60.0
 2021-01-28 13:10:00   3.0  6.0    60.0
 2021-01-28 13:11:00   3.0  6.0    58.0
 2021-01-28 13:12:00   3.0  6.0    56.0
 2021-01-28 13:13:00   3.0  